In [1]:
# Setup the Jupyter version of Dash
from dash import Dash
from dash import dcc, html


# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from bson import ObjectId

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CRUD module for accessing the Animal Shelter database
from CRUDMongo_1 import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Setup MongoDB connection credentials
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
CRUDMongo_1 = AnimalShelter(username, password)

# Get data from the database and convert it inot a pandas Dataframe##################################################
# df = pd.DataFrame.from_records(CRUDMongo_1.read({}))
df = pd.read_csv("aac_shelter_outcomes.csv")  # load file into a pandas Dataframe

print(df.head()) # show the first five rows of dataframe

# Convert the DataFrame into a list of dictionaries (one dictionary per row)
data_to_insert = df.to_dict(orient='records')

# Check the first few records to make sure they are in the correct format
print(f"Data to insert (first record): {data_to_insert[0]}")

# Insert each record into MongoDB using the CRUDMongo_1 'create' method
for record in data_to_insert:
    CRUDMongo_1.create(record)  # Insert each record into the MongoDB collection

print("Data has been successfully inserted into MongoDB.")

# Fetch all data from MongoDB as a test
data_from_db = CRUDMongo_1.read({})

# Print the fetched data
print(f"Data fetched from MongoDB (first 5 records): {data_from_db[:5]}")
#
################################################################################################################

# print(df.columns)

# Ensure that the '_id' column exists before trying to drop it
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)



#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Joseph Szabo, CS-340 Dashboard'))),
    html.Hr(),
    #html.Div(
        
    # Add Radio Buttons in code for the interactive filtering options.
    dcc.RadioItems(
        id='filter-type', 
        # created radio button labels 
        options=[
            {'label': 'All Animals', 'value': 'RESET'},
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster/Individual Tracking', 'value': 'DIT'},
        ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable = "single", # allows a row to be selected
        editable=True,
        selected_rows=[0],
        page_action="native", # enable native page transition
        sort_action="native", # Allow for sorting
        filter_action="native", # Allow filter options
        page_current=0, #set the start page = 0
        page_size=10, # Set 10 rows per page                 
    ),
    html.Br(),
    html.Hr(),
    
    #Set up the dashboard so that the chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the table basaed on the selected filter
@app.callback([Output('datatable-id','data'),
    Output('datatable-id','columns')],
    [Input('filter-type','value')])

def update_dashboard(filter_type):
    
    # Function to convert ObjectId to string for serialization 
    def convert_objectid_to_str(record):
        for key, value in record.items():
            if isinstance(value, ObjectId):
                record[key] = str(value)
                
            # Keep latitude and longitude numeric
            if key == 'location_lat' or key == 'location_long':
                try:
                    record[key] = float(value) # Conversion to float
                except ValueError:
                    record[key] = None # Handle potential errors with conversion
        return record
    
    #Default value returns all animals
    if filter_type == 'RESET':
     
        df = pd.DataFrame.from_records(CRUDMongo_1.read({})) # Get all data 
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        #convert ObjectIds to strings 
        data = [convert_objectid_to_str(record) for record in data]
        
        return data,columns
                 
    # Water Rescue Filter
    elif filter_type == 'WR':
        df = pd.DataFrame(list(CRUDMongo_1.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Female'},
                {'$or': [
                    {'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesapeake Bay Retriever'},
                    {'breed': 'Newfoundland Mix'},
                    {'breed': 'Newfoundland Australian Cattle Dog'},
                    {'breed': 'Newfoundland/Great Pyrenees'},
                    {'breed': 'Newfoundland/Labrador Retriever'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        })))
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        #convert ObjectIds to strings 
        data = [convert_objectid_to_str(record) for record in data]
        return data,columns
        
    # Mountain/Wilderness Rescue Filter
    elif filter_type == 'MWR':
        df = pd.DataFrame(list(CRUDMongo_1.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'$or': [
                    {'breed': 'German Shepherd'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        })))
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        #convert ObjectIds to strings
        data = [convert_objectid_to_str(record) for record in data]
        return data,columns
        
    # Disaster/Individual tracking filter
    elif filter_type == 'DIT':
        df = pd.DataFrame(list(CRUDMongo_1.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 20}},
                    {'age_upon_outcome_in_weeks': {'$lte': 300}}
                ]}
            ]
        })))
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        #convert ObjectIds to strings
        data = [convert_objectid_to_str(record) for record in data]
        return data,columns
    
    #If no condition matches
    return [], []

# Display a pie chart based on selected data from the table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len (viewData) == 0:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    
    # Add code for pie chart of animal breed
    return [
        dcc.Graph(            
            figure = px.pie(
                dff, 
                names='breed',
                title='Preferred Animals'
            )
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    #Check if selected_columns are empty
    if selected_columns is None or len(selected_columns) == 0:
        return []
    
    # Highlight selected column
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Callback to display the map with the selected animal's location
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    # Convert selected data into DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    if '_id' in dff.columns:
        dff = dff.drop(columns=['_id'])
    
    
    # Select row based on user selection
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    lat = dff.iloc[row,13]
    long = dff.iloc[row,14]
    print(f"Debug: selected row: {row}, lat = {lat}, long = {long}")
        
    # Handle invalid lat and long values
    try:
        lat = float(lat)
        long = float(long)
        
    except (ValueError, IndexError):
        lat, long = None, None # Handle invalid or missing lat or long
                      
    #return empty string if lat or long are invalid.
    if lat is None or long is None:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, long], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

#app.run_server(mode='inline', debug=True)



Connected successfully to MongoDB!
   Unnamed: 0 age_upon_outcome animal_id animal_type                    breed  \
0           1          3 years   A746874         Cat   Domestic Shorthair Mix   
1           2           1 year   A725717         Cat   Domestic Shorthair Mix   
2           3          2 years   A716330         Dog  Chihuahua Shorthair Mix   
3           4         7 months   A733653         Cat              Siamese Mix   
4           5          2 years   A691584         Dog   Labrador Retriever Mix   

          color date_of_birth             datetime            monthyear  \
0   Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
1  Silver Tabby    2015-05-02  2016-05-06 10:49:00  2016-05-06T10:49:00   
2   Brown/White    2013-11-18  2015-12-28 18:43:00  2015-12-28T18:43:00   
3    Seal Point    2016-01-25  2016-08-27 18:11:00  2016-08-27T18:11:00   
4     Tan/White    2012-11-06  2015-05-30 13:48:00  2015-05-30T13:48:00   

    name outcome_subtype   

In [2]:
app.run(debug=True)

In [5]:
from CRUDMongo_1 import AnimalShelter
test = AnimalShelter("aacuser", "SNHU1234")
animal_data = {
    "animal_id": "L123456",
    "animal_type": "Dog",
    "breed": "Collie",
    "color": "Black",
    "name": "Lassie"
}
create_result = test.create(animal_data)
print(f"Document inserted: {create_result}") # Expected true, indicating succesful Insertion

read_data = {"name": "Lassie"}
read_result = test.read(read_data)
print(f" Results: {read_result}") # expect to print Lassies results

animal_data = {"name": "Lassie"}
new_data = {"color": "Brown, White"}
update_result = test.update(animal_data, new_data)
print(f"Updated Results: {update_result}") # Expect lassie iwth brown and white coloring.

read_updated_animals = test.read(new_data)
print(f"Update results: {read_updated_animals}") #Expected  lassies coloring will be brown and white

read_result = test.read({"name": "Lassie"})
print(f"Read result: {read_result}")

delete_data = {"name": "Lassie"}
delete_result = test.delete(delete_data)
print(f"Documents deleted: {delete_result}") # expeted to see 1 deleted result

Connected successfully to MongoDB!
Document inserted: True
 Results: [{'_id': ObjectId('6804172984a4cb8893f4a7cb'), 'animal_id': 'L123456', 'animal_type': 'Dog', 'breed': 'Collie', 'color': 'Black', 'name': 'Lassie'}]
Updated Results: 1
Update results: [{'_id': ObjectId('6804172984a4cb8893f4a7cb'), 'animal_id': 'L123456', 'animal_type': 'Dog', 'breed': 'Collie', 'color': 'Brown, White', 'name': 'Lassie'}]
Read result: [{'_id': ObjectId('6804172984a4cb8893f4a7cb'), 'animal_id': 'L123456', 'animal_type': 'Dog', 'breed': 'Collie', 'color': 'Brown, White', 'name': 'Lassie'}]
Documents deleted: 1
